# Load Submission Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Standard libraries
import os
import glob
import time
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

# Data handling and manipulation
import numpy as np
import pandas as pd

# Model saving/loading
import joblib

# Get the number of CPU threads available
num_threads = multiprocessing.cpu_count()
print(f"Number of CPU threads available: {num_threads}")

# Filepaths
PREDICT_PATH = '/content/drive/MyDrive/building_classification/submission.csv'

SAMPLE_SUBMISSION_PATH = '/content/drive/MyDrive/building_classification/sample_submission_v0.1.0.csv.gz'
OUTPUT_SUBMISSION_PATH = '/content/drive/MyDrive/building_classification/submission_postprocessed.csv'

test_preds = pd.read_csv(PREDICT_PATH)

Mounted at /content/drive
Number of CPU threads available: 8


In [ ]:
# display full columns
pd.set_option('display.max_columns', None)
test_preds.head()

,filename,Active_Power_Sensor,Air_Flow_Sensor,Air_Flow_Setpoint,Air_Temperature_Sensor,Air_Temperature_Setpoint,Alarm,Angle_Sensor,Average_Zone_Air_Temperature_Sensor,Chilled_Water_Differential_Temperature_Sensor,Chilled_Water_Return_Temperature_Sensor,Chilled_Water_Supply_Flow_Sensor,Chilled_Water_Supply_Temperature_Sensor,Command,Cooling_Demand_Sensor,Cooling_Demand_Setpoint,Cooling_Supply_Air_Temperature_Deadband_Setpoint,Cooling_Temperature_Setpoint,Current_Sensor,Damper_Position_Sensor,Damper_Position_Setpoint,Demand_Sensor,Dew_Point_Setpoint,Differential_Pressure_Sensor,Differential_Pressure_Setpoint,Differential_Supply_Return_Water_Temperature_Sensor,Discharge_Air_Dewpoint_Sensor,Discharge_Air_Temperature_Sensor,Discharge_Air_Temperature_Setpoint,Discharge_Water_Temperature_Sensor,Duration_Sensor,Electrical_Power_Sensor,Energy_Usage_Sensor,Filter_Differential_Pressure_Sensor,Flow_Sensor,Flow_Setpoint,Frequency_Sensor,Heating_Demand_Sensor,Heating_Demand_Setpoint,Heating_Supply_Air_Temperature_Deadband_Setpoint,Heating_Temperature_Setpoint,Hot_Water_Flow_Sensor,Hot_Water_Return_Temperature_Sensor,Hot_Water_Supply_Temperature_Sensor,Humidity_Setpoint,Load_Current_Sensor,Low_Outside_Air_Temperature_Enable_Setpoint,Max_Air_Temperature_Setpoint,Min_Air_Temperature_Setpoint,Outside_Air_CO2_Sensor,Outside_Air_Enthalpy_Sensor,Outside_Air_Humidity_Sensor,Outside_Air_Lockout_Temperature_Setpoint,Outside_Air_Temperature_Sensor,Outside_Air_Temperature_Setpoint,Parameter,Peak_Power_Demand_Sensor,Position_Sensor,Power_Sensor,Pressure_Sensor,Rain_Sensor,Reactive_Power_Sensor,Reset_Setpoint,Return_Air_Temperature_Sensor,Return_Water_Temperature_Sensor,Room_Air_Temperature_Setpoint,Sensor,Setpoint,Solar_Radiance_Sensor,Speed_Setpoint,Static_Pressure_Sensor,Static_Pressure_Setpoint,Status,Supply_Air_Humidity_Sensor,Supply_Air_Static_Pressure_Sensor,Supply_Air_Static_Pressure_Setpoint,Supply_Air_Temperature_Sensor,Supply_Air_Temperature_Setpoint,Temperature_Sensor,Temperature_Setpoint,Thermal_Power_Sensor,Time_Setpoint,Usage_Sensor,Valve_Position_Sensor,Voltage_Sensor,Warmest_Zone_Air_Temperature_Sensor,Water_Flow_Sensor,Water_Temperature_Sensor,Water_Temperature_Setpoint,Wind_Direction_Sensor,Wind_Speed_Sensor,Zone_Air_Dewpoint_Sensor,Zone_Air_Humidity_Sensor,Zone_Air_Humidity_Setpoint,Zone_Air_Temperature_Sensor
0,test_X20367.pkl,1.124618e-05,6.174588e-02,0.000103,0.002517,0.002165,0.000121,0.000018,0.000029,0.000019,1.007603e-06,0.000016,1.083975e-04,0.002211,0.0,0.000009,0.000004,8.526293e-07,0.046993,0.005860,1.011029e-07,0.281845,0.000019,0.000015,0.000004,0.000244,0.000035,0.000030,0.000165,0.000019,0.001383,0.121134,6.717472e-07,0.000005,0.008753,0.000014,0.000211,0.0,1.507825e-06,0.000009,0.000037,0.000016,0.000003,0.000003,0.000002,0.006768,0.000051,0.000026,0.000016,1.530039e-06,0.000063,0.000004,6.474234e-07,0.000017,0.000005,0.000348,0.002188,0.010240,0.063758,0.000480,0.000090,3.256702e-06,5.487293e-07,0.000105,3.048635e-04,0.000010,0.118712,0.002864,0.000021,0.000019,0.000017,0.000002,0.002243,0.000005,0.000005,9.354834e-07,0.000006,0.000630,0.013243,0.001199,2.842422e-05,1.832380e-07,0.000059,2.204660e-05,0.000288,0.000097,1.277554e-04,0.000052,0.000046,0.000007,0.000014,0.001010,0.000150,0.000011,0.000178
1,test_X103084.pkl,1.571470e-06,6.911209e-07,0.000015,0.000002,0.000103,0.000019,0.000052,0.000006,0.000004,3.571034e-06,0.999809,8.751079e-07,0.000106,0.0,0.000017,0.000069,9.700913e-06,0.000003,0.000001,7.514865e-07,0.035091,0.000009,0.000003,0.000242,0.000008,0.000016,0.000003,0.000230,0.000001,0.000505,0.013493,4.598089e-03,0.000009,0.474578,0.000058,0.000004,0.0,1.119816e-05,0.000115,0.000004,0.001553,0.000008,0.000015,0.000003,0.000008,0.000388,0.000116,0.000066,2.297328e-06,0.000033,0.000009,1.945684e-04,0.000004,0.000025,0.000227,0.000902,0.000009,0.232576,0.000054,0.000033,5.049494e-07,2.011498e-06,0.000010,3.411090e-05,0.000017,0.984755,0.000368,0.000342,0.001364,0.000029,0.000191,0.000081,0.000052,0.000097,7.2

In [ ]:
# make a copy of predictions so we dont have to rerun predict when testing post predictions changes
test_preds_copy = test_preds.copy()

# drop filename from test_preds_copy
test_preds_copy = test_preds_copy.drop(columns=['filename'])

In [ ]:
test_preds_copy.shape

(315720, 94)

# Post processing

In [ ]:
import collections

hierarchy = {
    'Sensor': [
        'Angle_Sensor',
        'Outside_Air_CO2_Sensor',
        'Current_Sensor',
        'Demand_Sensor',
        'Wind_Direction_Sensor',
        'Zone_Air_Dewpoint_Sensor',
        'Discharge_Air_Dewpoint_Sensor',
        'Duration_Sensor',
        'Outside_Air_Enthalpy_Sensor',
        'Energy_Usage_Sensor',
        'Flow_Sensor',
        'Frequency_Sensor',
        'Outside_Air_Humidity_Sensor',
        'Zone_Air_Humidity_Sensor',
        'Supply_Air_Humidity_Sensor',
        'Position_Sensor',
        'Power_Sensor',
        'Pressure_Sensor',
        'Rain_Sensor',
        'Voltage_Sensor',
        'Solar_Radiance_Sensor',
        'Wind_Speed_Sensor',
        'Temperature_Sensor',
        'Usage_Sensor',
    ],
    'Current_Sensor': ['Load_Current_Sensor'],
    'Demand_Sensor': ['Cooling_Demand_Sensor', 'Heating_Demand_Sensor', 'Peak_Power_Demand_Sensor'],
    'Flow_Sensor': ['Air_Flow_Sensor', 'Water_Flow_Sensor'],
    'Water_Flow_Sensor': ['Hot_Water_Flow_Sensor', 'Chilled_Water_Supply_Flow_Sensor'],
    'Position_Sensor': ['Damper_Position_Sensor', 'Valve_Position_Sensor'],
    'Power_Sensor': ['Electrical_Power_Sensor', 'Thermal_Power_Sensor'],
    'Electrical_Power_Sensor': ['Active_Power_Sensor', 'Reactive_Power_Sensor'],
    'Pressure_Sensor': ['Static_Pressure_Sensor', 'Differential_Pressure_Sensor'],
    'Static_Pressure_Sensor': ['Supply_Air_Static_Pressure_Sensor'],
    'Differential_Pressure_Sensor': ['Filter_Differential_Pressure_Sensor'],
    'Temperature_Sensor': ['Water_Temperature_Sensor', 'Air_Temperature_Sensor'],
    'Water_Temperature_Sensor': [
        'Hot_Water_Supply_Temperature_Sensor',
        'Chilled_Water_Differential_Temperature_Sensor',
        'Return_Water_Temperature_Sensor',
    ],
    'Return_Water_Temperature_Sensor': [
        'Hot_Water_Return_Temperature_Sensor',
        'Chilled_Water_Return_Temperature_Sensor',
        'Differential_Supply_Return_Water_Temperature_Sensor',
    ],
    'Air_Temperature_Sensor': [
        'Return_Air_Temperature_Sensor',
        'Discharge_Air_Temperature_Sensor',
        'Supply_Air_Temperature_Sensor',
        'Outside_Air_Temperature_Sensor',
    ],
    'Outside_Air_Temperature_Sensor': ['Zone_Air_Temperature_Sensor'],
    'Zone_Air_Temperature_Sensor': ['Average_Zone_Air_Temperature_Sensor', 'Warmest_Zone_Air_Temperature_Sensor'],
    'Setpoint': [
        'Damper_Position_Setpoint',
        'Cooling_Demand_Setpoint',
        'Heating_Demand_Setpoint',
        'Dew_Point_Setpoint',
        'Flow_Setpoint',
        'Humidity_Setpoint',
        'Reset_Setpoint',
        'Differential_Pressure_Setpoint',
        'Static_Pressure_Setpoint',
        'Speed_Setpoint',
        'Temperature_Setpoint',
        'Time_Setpoint',
    ],
    'Flow_Setpoint': ['Air_Flow_Setpoint'],
    'Humidity_Setpoint': ['Zone_Air_Humidity_Setpoint'],
    'Static_Pressure_Setpoint': ['Supply_Air_Static_Pressure_Setpoint'],
    'Temperature_Setpoint': [
        'Air_Temperature_Setpoint',
        'Cooling_Temperature_Setpoint',
        'Heating_Temperature_Setpoint',
        'Water_Temperature_Setpoint',
    ],
    'Air_Temperature_Setpoint': [
        'Discharge_Air_Temperature_Setpoint',
        'Max_Air_Temperature_Setpoint',
        'Min_Air_Temperature_Setpoint',
        'Outside_Air_Temperature_Setpoint',
    ],
    'Outside_Air_Temperature_Setpoint': [
        'Outside_Air_Lockout_Temperature_Setpoint',
        'Low_Outside_Air_Temperature_Enable_Setpoint',
    ],
    'Cooling_Temperature_Setpoint': ['Cooling_Supply_Air_Temperature_Deadband_Setpoint'],
    'Heating_Temperature_Setpoint': ['Heating_Supply_Air_Temperature_Deadband_Setpoint'],
}

In [ ]:
# --------------------------------------------------------------------
# HELPER FUNCTIONS
# --------------------------------------------------------------------
def zero_out_subtree(label, pred_dict, hierarchy):
    """Recursively set label and all descendants to 0 in pred_dict."""
    pred_dict[label] = 0.0
    for child in hierarchy.get(label, []):
        zero_out_subtree(child, pred_dict, hierarchy)

def top_down_prune(label, pred_dict, hierarchy, threshold=0.4):
    """
    If the parent's probability < threshold, recursively set parent's subtree to 0.
    Otherwise, recurse on its children.
    """
    if pred_dict.get(label, 0.0) < threshold:
        zero_out_subtree(label, pred_dict, hierarchy)
    else:
        for child in hierarchy.get(label, []):
            top_down_prune(child, pred_dict, hierarchy, threshold=threshold)

def compute_subtree_sum_and_count(label, pred_dict, hierarchy):
    """Return (sum_of_probs_in_subtree, count_of_nodes_in_subtree)."""
    total_sum, total_count = 0.0, 0
    stack = [label]
    while stack:
        node = stack.pop()
        total_sum += pred_dict.get(node, 0.0)
        total_count += 1
        children = hierarchy.get(node, [])
        for c in children:
            stack.append(c)
    return total_sum, total_count

def level_wise_winner(label, pred_dict, hierarchy):
    """
    1) For each child of `label`, compute the average prob in its subtree.
    2) Keep only the child with highest average (zero-out siblings).
    3) Recurse on that winner child.
    """
    # If parent is zero, no need to proceed (already pruned).
    if pred_dict.get(label, 0.0) == 0.0:
        return

    children = hierarchy.get(label, [])
    if not children:
        return  # No children -> nothing to do

    # Compute subtree averages for each child
    child_avgs = []
    for child in children:
        subtree_sum, subtree_count = compute_subtree_sum_and_count(child, pred_dict, hierarchy)
        avg_val = subtree_sum / subtree_count if subtree_count > 0 else 0.0
        child_avgs.append((child, avg_val))

    # Find the child with the maximum average
    winner_child, winner_avg = max(child_avgs, key=lambda x: x[1])

    # Zero out the losing children
    for (child, _) in child_avgs:
        if child != winner_child:
            zero_out_subtree(child, pred_dict, hierarchy)

    # Recurse on the winner
    level_wise_winner(winner_child, pred_dict, hierarchy)

def run_second_pass_level_wise(pred_dict, hierarchy, top_level_labels):
    """Apply level_wise_winner(...) from each top-level root to its subtree."""
    for root in top_level_labels:
        level_wise_winner(root, pred_dict, hierarchy)

In [ ]:
# --------------------------------------------------------------------
# MAIN POST-PROCESSING FUNCTION
# --------------------------------------------------------------------
def two_pass_postprocess(df, hierarchy, threshold=0.4):
    """
    1) Identify top-level labels.
    2) For each row in df, do the top-down pass + second pass.
    3) Return a new DataFrame with updated probabilities.
    """

    # Identify top-level labels (some are those that are in hierarchy but never appear as children)
    # We'll also allow user to define certain known top-level labels if you have them.
    all_children = set()
    for parent, kids in hierarchy.items():
        for k in kids:
            all_children.add(k)

    # By default, consider any label in `hierarchy` that is not a child as top-level
    dict_toplevel = [lbl for lbl in hierarchy.keys() if lbl not in all_children]

    # You might have additional top-level labels that appear in the columns but not in hierarchy at all
    # (like "Alarm", "Command", "Parameter", "Status", etc.)
    # Let's merge them:
    possible_toplevel = {"Alarm", "Command", "Parameter", "Status"}  # or any known top-level labels
    #possible_toplevel = {}
    top_level_labels = set(dict_toplevel).union(possible_toplevel)

    # Intersect with df columns to ensure we only keep relevant columns
    top_level_labels = list(top_level_labels.intersection(df.columns))

    # Create a new DataFrame for the results (or modify in place)
    df_post = df.copy()

    # Go row by row
    for i in range(len(df_post)):
        # Convert row to dict: label -> prob
        row_dict = df_post.iloc[i].to_dict()

        # ========== PASS 1: Top-down prune ==========
        for root_label in top_level_labels:
            if root_label in row_dict:
                top_down_prune(root_label, row_dict, hierarchy, threshold=threshold)

        # ========== PASS 2: Level-wise winner ==========
        run_second_pass_level_wise(row_dict, hierarchy, top_level_labels)

        # Write updated probabilities back into df_post
        for col in row_dict:
            if col in df_post.columns:
                df_post.at[i, col] = row_dict[col]

    return df_post

In [ ]:
test_preds_copy = two_pass_postprocess(test_preds_copy, hierarchy, threshold=0.3)

# Checks and save

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# insert "filename" column from test_preds into test_preds_copy
test_preds_copy.insert(0, 'filename', test_preds['filename'])

# Display the first few rows
test_preds_copy.head()

,filename,Active_Power_Sensor,Air_Flow_Sensor,Air_Flow_Setpoint,Air_Temperature_Sensor,Air_Temperature_Setpoint,Alarm,Angle_Sensor,Average_Zone_Air_Temperature_Sensor,Chilled_Water_Differential_Temperature_Sensor,Chilled_Water_Return_Temperature_Sensor,Chilled_Water_Supply_Flow_Sensor,Chilled_Water_Supply_Temperature_Sensor,Command,Cooling_Demand_Sensor,Cooling_Demand_Setpoint,Cooling_Supply_Air_Temperature_Deadband_Setpoint,Cooling_Temperature_Setpoint,Current_Sensor,Damper_Position_Sensor,Damper_Position_Setpoint,Demand_Sensor,Dew_Point_Setpoint,Differential_Pressure_Sensor,Differential_Pressure_Setpoint,Differential_Supply_Return_Water_Temperature_Sensor,Discharge_Air_Dewpoint_Sensor,Discharge_Air_Temperature_Sensor,Discharge_Air_Temperature_Setpoint,Discharge_Water_Temperature_Sensor,Duration_Sensor,Electrical_Power_Sensor,Energy_Usage_Sensor,Filter_Differential_Pressure_Sensor,Flow_Sensor,Flow_Setpoint,Frequency_Sensor,Heating_Demand_Sensor,Heating_Demand_Setpoint,Heating_Supply_Air_Temperature_Deadband_Setpoint,Heating_Temperature_Setpoint,Hot_Water_Flow_Sensor,Hot_Water_Return_Temperature_Sensor,Hot_Water_Supply_Temperature_Sensor,Humidity_Setpoint,Load_Current_Sensor,Low_Outside_Air_Temperature_Enable_Setpoint,Max_Air_Temperature_Setpoint,Min_Air_Temperature_Setpoint,Outside_Air_CO2_Sensor,Outside_Air_Enthalpy_Sensor,Outside_Air_Humidity_Sensor,Outside_Air_Lockout_Temperature_Setpoint,Outside_Air_Temperature_Sensor,Outside_Air_Temperature_Setpoint,Parameter,Peak_Power_Demand_Sensor,Position_Sensor,Power_Sensor,Pressure_Sensor,Rain_Sensor,Reactive_Power_Sensor,Reset_Setpoint,Return_Air_Temperature_Sensor,Return_Water_Temperature_Sensor,Room_Air_Temperature_Setpoint,Sensor,Setpoint,Solar_Radiance_Sensor,Speed_Setpoint,Static_Pressure_Sensor,Static_Pressure_Setpoint,Status,Supply_Air_Humidity_Sensor,Supply_Air_Static_Pressure_Sensor,Supply_Air_Static_Pressure_Setpoint,Supply_Air_Temperature_Sensor,Supply_Air_Temperature_Setpoint,Temperature_Sensor,Temperature_Setpoint,Thermal_Power_Sensor,Time_Setpoint,Usage_Sensor,Valve_Position_Sensor,Voltage_Sensor,Warmest_Zone_Air_Temperature_Sensor,Water_Flow_Sensor,Water_Temperature_Sensor,Water_Temperature_Setpoint,Wind_Direction_Sensor,Wind_Speed_Sensor,Zone_Air_Dewpoint_Sensor,Zone_Air_Humidity_Sensor,Zone_Air_Humidity_Setpoint,Zone_Air_Temperature_Sensor
0,test_X20367.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.083975e-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000019,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000010,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,test_X103084.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999809,8.751079e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000001,0.0,0.0,0.0,0.0,0.474578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000017,0.984755,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.975702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,test_X6910.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.507888e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000117,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,test_X66332.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.465029e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.347399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000067,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000003,0.9

In [ ]:
# Save Submission
print(f"Saving submission to {OUTPUT_SUBMISSION_PATH}...")
test_preds_copy.to_csv(OUTPUT_SUBMISSION_PATH, index=False)
print(f"Submission saved to {OUTPUT_SUBMISSION_PATH}")

Saving submission to /content/drive/MyDrive/building_classification/submission_postprocessed.csv...
Submission saved to /content/drive/MyDrive/building_classification/submission_postprocessed.csv
